## Foundation Model - AlexTM (In-context Learning)

#### Imports 

In [3]:
from sagemaker.utils import name_from_base
from sagemaker.predictor import Predictor
from sagemaker import get_execution_role
from sagemaker import hyperparameters
from sagemaker.model import Model
from sagemaker import script_uris
from sagemaker import image_uris 
from sagemaker import model_uris
from datetime import datetime
import sagemaker
import json

#### Setup logging

#### Log versions of dependencies 

In [ ]:



# model_version = "*" fetches the latest version of the model
model_id, model_version = "pytorch-textgeneration1-alexa20b", "*"

endpoint_name = name_from_base(f"jumpstart-example-infer-{model_id}")

endpoint_config_name = "config-" + endpoint_name


In [ ]:
aws_role = get_execution_role()